In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import minmax_scale

In [3]:
cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
orig_cols = ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate','clean_comment']
train = pd.read_csv('../train_meta.csv')

In [4]:
train.shape

(159571, 269)

In [5]:
test = pd.read_csv('../test_meta.csv')
test.shape

(153164, 263)

In [6]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score,log_loss

stacker = lgb.LGBMClassifier(max_depth=4, metric="auc", n_estimators=90, num_leaves=9, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.4, colsample_bytree=0.4, bagging_fraction=0.8, bagging_freq=5, reg_lambda=10.2)

sub = pd.read_csv('../input/sample_submission.csv')
for c in cols:
    sub[c] = 0

scores = []
for label in cols:
    print(label)
    kf = KFold(n_splits=10, shuffle=True, random_state=1256)
    c_score = []
    for train_index, test_index in kf.split(train):
        stacker.fit(train.drop(orig_cols, axis=1).iloc[train_index,:], train[label][train_index])
        score = roc_auc_score(train[label][test_index], stacker.predict_proba(train.drop(orig_cols, axis=1).iloc[test_index,:])[:,1])
        print("AUC:", score)    
        scores.append(score)
        c_score.append(score)
        sub[label] += minmax_scale(stacker.predict_proba(test.drop(['id','comment_text','clean_comment'],axis=1))[:,1])*0.1
    print(label,np.mean(c_score))
print("CV score:", np.mean(scores))

toxic
AUC: 0.9899130757581646
AUC: 0.990396659518011
AUC: 0.9893189749873188
AUC: 0.9901955337416918
AUC: 0.9900996385195607
AUC: 0.9897545506859042
AUC: 0.9905374357225498
AUC: 0.9890456601252697
AUC: 0.9909183512252722
AUC: 0.9895523405401543
toxic 0.9899732220823896
severe_toxic
AUC: 0.9920230590807871
AUC: 0.9931723771359539
AUC: 0.9924369374585343
AUC: 0.9937881964040958
AUC: 0.9907073039164446
AUC: 0.992363223843655
AUC: 0.9920593487425766
AUC: 0.9932111911341657
AUC: 0.9915504313472546
AUC: 0.9933215242192279
severe_toxic 0.9924633593282696
obscene
AUC: 0.9963279485363086
AUC: 0.9960808721633464
AUC: 0.9965764811362505
AUC: 0.9961726679420453
AUC: 0.9957607256092087
AUC: 0.9960230924194133
AUC: 0.9963302232412863
AUC: 0.9953402618681836
AUC: 0.9955918604012135
AUC: 0.9957518928174246
obscene 0.995995602613468
threat
AUC: 0.995767482020679
AUC: 0.9835470488402791
AUC: 0.9962462732138367
AUC: 0.9976921520911615
AUC: 0.9964488670713126
AUC: 0.9970992841758882
AUC: 0.990291743478017

In [7]:
sub.to_csv('stack.csv',index=False)

In [9]:
c_score

[0.9940805178728247,
 0.9931477335006713,
 0.9945980364707051,
 0.9947613990335862,
 0.993759053597296,
 0.9886412887942657,
 0.990405476566614,
 0.9934977605937998,
 0.9941031164727847,
 0.9874350682162708]